In [1]:
""" Creates batches of images to feed into the training network conditioned by genre, uses upsampling when creating batches to account for uneven distributions """

' Creates batches of images to feed into the training network conditioned by genre, uses upsampling when creating batches to account for uneven distributions '

In [2]:
import numpy as np
import imageio
import time
import random
import os
from pathlib import Path
from PIL import Image

In [3]:
import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# import settings

# Set the dimension of images you want to be passed in to the network
DIM = 64

In [4]:
# Set your own path to images
# src_img_path = os.path.normpath(settings.RESIZED_IMAGES_PATH)
src_img_path  = Path('/home/ec2-user/SageMaker/wikiart-portrait')

In [28]:
# This dictionary should be updated to hold the absolute number of images associated with each genre used during training
styles = {
    "portraits": 14980,
    "abstract": 0,
    "landscape": 0
}

styleNum = {
    "portraits": 0,
    "abstract": 1,
    "landscape": 2
}

curPos = {
    "portraits": 0,
    "abstract": 0,
    "landscape": 0
}

In [10]:
# # This dictionary should be updated to hold the absolute number of images associated with each genre used during training
# styles = {
#     "portraits": 14981
# }

# styleNum = {
#     "portraits": 0
# }

# curPos = {
#     "portraits": 0
# }

In [11]:
testNums = {}
trainNums = {} 

In [14]:
# Generate test set of images made up of 1/20 of the images (per genre)
for k, v in styles.items():
#     print(k,v)
    # put a twentieth of paintings in here
    nums = range(v)
    random.shuffle(list(nums))
    testNums[k] = nums[0 : v // 20]
    trainNums[k] = nums[v // 20 :]
    print(testNums[k])

range(0, 749)
range(0, 0)
range(0, 0)


In [32]:
def inf_gen(gen):
    while True:
        for (images, labels) in gen():
            yield images, labels

In [59]:
def make_generator(files, batch_size, n_classes):
    if batch_size % n_classes != 0:
        raise ValueError(
            "Batch size {} must be divisible by num classes {}".format(batch_size, n_classes)
        )

    class_batch = batch_size // n_classes

    generators = []

    def get_epoch():

        while True:

            images = np.zeros((batch_size, 3, DIM, DIM), dtype="int32")
            labels = np.zeros((batch_size, n_classes))
            n = 0
            for style in styles:
#                 print('omg')
                styleLabel = styleNum[style]
                curr = curPos[style]
                for _ in range(class_batch):
                    if curr == styles[style]:
                        curr = 0
                        random.shuffle(list(files[style]))

#                     img_path = Path(src_img_path, style, str(curr) + ".png")
#                     print(styleLabel, str(curr), 'yo')
                    img_path = str(Path(src_img_path, str(styleLabel), str(curr) + ".jpg"))
                    print(img_path)
                    image = Image.open(img_path).convert(mode="RGB")
                    image = np.asarray(image)

                    images[n % batch_size] = image.transpose(2, 0, 1)
                    labels[n % batch_size, int(styleLabel)] = 1
                    n += 1
                    curr += 1
                curPos[style] = curr

            # randomize things but keep relationship between a conditioning vector and its associated image
            rng_state = np.random.get_state()
            np.random.shuffle(images)
            np.random.set_state(rng_state)
            np.random.shuffle(labels)
            yield (images, labels)

    return get_epoch

In [15]:
def make_generator(files, batch_size, n_classes, genre):
    if batch_size % n_classes != 0:
        raise ValueError(
            "Batch size {} must be divisible by num classes {}".format(batch_size, n_classes)
        )

    class_batch = batch_size // n_classes

    generators = []

    def get_epoch():

        while True:

            images = np.zeros((batch_size, 3, DIM, DIM), dtype="int32")
            labels = np.zeros((batch_size, n_classes))
            n = 0
            for style in genre:
#                 print('omg')
                styleLabel = styleNum[style]
                curr = curPos[style]
                for _ in range(class_batch):
                    if curr == styles[style]:
                        curr = 0
                        random.shuffle(list(files[style]))

#                     img_path = Path(src_img_path, style, str(curr) + ".png")
#                     print(styleLabel, str(curr), 'yo')
                    img_path = str(Path(src_img_path, str(styleLabel), str(curr) + ".jpg"))
                    print(img_path)
                    image = Image.open(img_path).convert(mode="RGB")
                    image = np.asarray(image)

                    images[n % batch_size] = image.transpose(2, 0, 1)
                    labels[n % batch_size, int(styleLabel)] = 1
                    n += 1
                    curr += 1
                curPos[style] = curr

            # randomize things but keep relationship between a conditioning vector and its associated image
            rng_state = np.random.get_state()
            np.random.shuffle(images)
            np.random.set_state(rng_state)
            np.random.shuffle(labels)
            yield (images, labels)

    return get_epoch

In [21]:
genre = ['portraits']

In [22]:
def load(batch_size):
    return (
        make_generator(trainNums, batch_size, len(genre),genre),
        make_generator(testNums, batch_size, len(genre),genre),
    )

In [1]:
# # Testing code to validate that the logic in generating batches is working properly and quickly
# train_gen, valid_gen = load(2)
# t0 = time.time()
# for i, batch in enumerate(train_gen(), start=1):
#     try: 
#         a, b = batch
#         print("time ", str(time.time() - t0))
#         if i == 900:
#             break
#         t0 = time.time()
#     except e:
#         pass

In [27]:
'14980.jpg' in os.listdir('/home/ec2-user/SageMaker/wikiart-portrait/0')

False

In [25]:
len( os.listdir('/home/ec2-user/SageMaker/wikiart-portrait/0'))

14980

In [6]:
import os
os.getcwd()
collection = "/home/ec2-user/SageMaker/genre-224/portraits"
for i, filename in enumerate(os.listdir(collection)):
    os.rename(str(collection)+ '/' + filename, str(collection)+ '/' + str(i) + ".jpg")

In [7]:
lst = []
directory = os.listdir(collection)
for i in range(len(os.listdir(collection))):
    num = int(directory[i].split('.')[0])
#     print(num)
    lst.append(num)

In [8]:
lst.sort()

In [9]:
import shutil
# shutil.rmtree(collection)

In [10]:
len(lst)

14988